In [48]:
import os
import sys
sys.path.append('../utils')
os.makedirs('models', exist_ok=True)

import joblib
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

from preprocessing import load_cicids_csv, preprocess_cicids
from model import train_incremental_model, save_model

In [49]:
DATA_PATH = '../data/TrafficLabelling/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv'  
MODEL_OUTPUT_PATH = '../models/model_cicids.pt'              # model scikit-learn salvat cu joblib
LABEL_ENCODER_PATH = '../models/label_encoder_cicids.pkl'    # salveaza LabelEncoder
CLASSES_PATH = '../models/classes_cicids.pkl'

In [50]:
# Etichetare exemplu
labels = ["normal", "attack", "normal", "attack"]

# 1. Creezi obiectul
label_encoder = LabelEncoder()

# 2. Îl antrenezi
label_encoder.fit(labels)

# 3. Salvezi clasele în fișier
joblib.dump(label_encoder.classes_, CLASSES_PATH)
joblib.dump(X.columns.tolist(), "models/feature_names.pkl")


['models/feature_names.pkl']

In [51]:
# Încarcă și preprocesează CICIDS
df = load_cicids_csv(DATA_PATH)
df.columns = df.columns.str.strip()

print(df.columns)

print("Dimensiune df:", df.shape)

X, y = preprocess_cicids(df)
print("Tipul lui y:", type(y))
print("Primele 5 etichete:", y[:5])

Index(['Flow ID', 'Source IP', 'Source Port', 'Destination IP',
       'Destination Port', 'Protocol', 'Timestamp', 'Flow Duration',
       'Total Fwd Packets', 'Total Backward Packets',
       'Total Length of Fwd Packets', 'Total Length of Bwd Packets',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Min Packet Length', 'Max Packet Length', 'Packet Length Mean',
  

In [52]:
X = X.replace([np.inf, -np.inf], np.nan)

In [53]:
# Imputarea valorilor lipsă cu mediana
imputer = SimpleImputer(strategy='median')
X_imputed = imputer.fit_transform(X)

In [54]:
# Transformă în DataFrame 
X = pd.DataFrame(X_imputed, columns=X.columns)


In [55]:
# Verificare labeluri
print("Exemple de labeluri:", set(y))

Exemple de labeluri: {np.int64(0), np.int64(1)}


In [56]:
# Obține toate clasele unice – necesar pentru partial_fit inițial
classes = sorted(set(y))

In [57]:
# Antrenează modelul incremental
model = train_incremental_model(X, y, classes=classes)

In [58]:
# Creează directorul dacă nu există
os.makedirs(os.path.dirname(MODEL_OUTPUT_PATH), exist_ok=True)

In [59]:
# Salvează modelul
save_model(model, MODEL_OUTPUT_PATH)

In [60]:
# Salvează și encoderul labelurilor dacă l-ai folosit separat (aici presupunem că y este deja numeric)
joblib.dump(classes, LABEL_ENCODER_PATH)

['../models/label_encoder_cicids.pkl']

In [61]:
print(f"Modelul antrenat pe CICIDS a fost salvat la: {MODEL_OUTPUT_PATH}")

Modelul antrenat pe CICIDS a fost salvat la: ../models/model_cicids.pt
